In [1]:
import numpy as np
import torch
torch.set_printoptions(edgeitems=2, threshold=50)

/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
%pip install imageio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 17.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
import imageio

img_arr = imageio.imread('../data/p1ch4/image-dog/bobby.jpg')
img_arr.shape

/tmp/ipykernel_31818/1529188539.py:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_arr = imageio.imread('../data/p1ch4/image-dog/bobby.jpg')


(720, 1280, 3)

In [9]:
img = torch.from_numpy(img_arr)
out = img.permute(2, 0, 1)

In [10]:
batch_size = 3
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)

In [18]:
import os

data_dir = '../data/p1ch4/image-cats/'
filenames = [name for name in os.listdir(data_dir)
             if os.path.splitext(name)[-1] == '.png']
for i, filename in enumerate(filenames):
    img_arr = imageio.imread(os.path.join(data_dir, filename))
    img_t = torch.from_numpy(img_arr).permute(2, 0, 1)[:3]
    batch[i] = img_t

/tmp/ipykernel_31818/2230534878.py:7: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_arr = imageio.imread(os.path.join(data_dir, filename))


In [19]:
batch.shape

torch.Size([3, 3, 256, 256])

In [32]:
batch = batch.float()
batch = batch / 255.0

In [22]:
# First way

""""n_channels = batch.shape[1]
for c in range(n_channels):
    mean = torch.mean(batch[:, c])
    std = torch.std(batch[:, c])
    batch[:, c] = (batch[:, c] - mean) / std"""

In [34]:
batch.shape, batch[:, 0].shape

(torch.Size([3, 3, 256, 256]), torch.Size([3, 256, 256]))

In [40]:
n_channels = batch.shape[1]

means = torch.zeros(n_channels, dtype = torch.float32) 
stds = torch.zeros(n_channels, dtype=torch.float32)
for c in range(n_channels):
    means[c] = torch.mean(batch[:, c])
    stds[c] = torch.std(batch[:, c])

means, stds

(tensor([0.5799, 0.4493, 0.3554]), tensor([0.2212, 0.2068, 0.1931]))

In [35]:
from torchvision.transforms import Normalize

batch_normalized = torch.zeros(batch.shape)
means = torch.zeros(n_channels, dtype = torch.float32) 
stds = torch.zeros(n_channels, dtype=torch.float32)

n_channels = batch.shape[1]

for c in range(n_channels):
    means[c] = torch.mean(batch[:, c])
    stds[c] = torch.std(batch[:, c])

batch_normalized = Normalize(means, stds)(batch)

In [41]:
batch_normalized[0,1,:,:].mean(), batch_normalized[0,1,:,:].std() # Zero mean and unit std

(tensor(0.0408), tensor(0.8636))